In [ ]:
import cv2
import os
import rosbag2_py
import datetime
import json

from matplotlib import pyplot as plt
from matplotlib.figure import Figure
from matplotlib import gridspec
import pandas as pd
import numpy as np
from tqdm import tqdm

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from deepracer_interfaces_pkg.msg import InferResultsArray, InferResults
from rclpy.serialization import deserialize_message

from deepracer_viz.gradcam.cam import GradCam
from deepracer_viz.model.model import Model
from deepracer_viz.model.metadata import ModelMetadata


In [ ]:
bridge = CvBridge()

In [ ]:
def get_rosbag_options(path, serialization_format='cdr'):
    storage_options = rosbag2_py.StorageOptions(uri=path, storage_id='sqlite3')

    converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format=serialization_format,
        output_serialization_format=serialization_format)

    return storage_options, converter_options

In [ ]:
def create_plot(action_names, height, width, dpi):
   
    fig = plt.figure(figsize=(width/dpi,height/dpi), dpi=dpi)
    x = list(range(0,len(action_names)))

    spec = gridspec.GridSpec(ncols=2, nrows=2,
                            width_ratios=[1, 1], wspace=0.1,
                            hspace=0.1, height_ratios=[3, 1], left=0.05,right=0.95,top=0.99,bottom=0.2)
    ax0 = fig.add_subplot(spec[0])
    plt.setp(ax0, xlabel="Original")
    plt.tick_params(labelleft = False, labelbottom=False)
    ax1 = fig.add_subplot(spec[1])
    plt.setp(ax1, xlabel="GradCam")
    plt.tick_params(labelleft = False, labelbottom=False)

    ax2 = fig.add_subplot(spec[2])
    ax2.set_ylim(0.0, 1.0)
    plt.setp(ax2, xlabel="OpenVINO")
    plt.xticks(x,action_names[::-1],rotation='vertical')

    ax3 = fig.add_subplot(spec[3], sharey=ax2)
    ax3.set_ylim(0.0, 1.0)
    plt.setp(ax3, xlabel="Tensorflow")
    plt.xticks(x,action_names[::-1],rotation='vertical')
    fig.canvas.draw()

    return fig

In [ ]:
def update_plot(fig: Figure, step, action_names, img, gradcam):

    x = list(range(0,len(action_names)))

    openvino_result = pd.DataFrame(step['openvino_results'])
    tf_result = pd.DataFrame(step['tf_results'])

    ax = fig.get_axes()

    for a in ax:
        for p in set(a.containers):
            p.remove()
        for i in set(a.images):
            i.remove()

    ax[0].imshow(img)
    ax[1].imshow(gradcam)
    ax[2].bar(x,openvino_result['probability'][::-1], color='blue')
    ax[3].bar(x,tf_result['probability'][::-1], color='blue')
    fig.canvas.draw()

    buf = fig.canvas.buffer_rgba()
    ncols, nrows = fig.canvas.get_width_height()
    return cv2.cvtColor(np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 4), cv2.COLOR_RGBA2BGR)


In [ ]:
root_path = '/workspaces/deepracer-devel/'
bag_path = os.path.join(root_path, 'output/garage_logs_1/duckworth-2023-reInvent-TTP-I-1-4-20240428-155457')
model_path = os.path.join(root_path, 'models/2023-reInvent-TTP-I-1-4')
metadata_json = os.path.join(model_path,'model_metadata.json')
model_pb = os.path.join(model_path,'agent/model.pb')

CODEC = "avc1"
output_file = "{}.mp4".format(bag_path)
width = 900
height = 600
fps = 30

In [ ]:
storage_options, converter_options = get_rosbag_options(bag_path)
storage_filter = rosbag2_py.StorageFilter(topics=['/inference_pkg/rl_results'])

reader = rosbag2_py.SequentialReader()
reader.open(storage_options, converter_options)

metadata = ModelMetadata.from_file(metadata_json)
model = Model.from_file(model_pb_path=model_pb, metadata=metadata, log_device_placement=False)

In [ ]:
reader.set_filter(storage_filter)

first_stamp = -1
steps_data = {'steps': []}

s = 0

while reader.has_next() and s < 60:
    step = {}
    
    (topic, data, t) = reader.read_next()
    msg = deserialize_message(data, InferResultsArray)

    # Timestamp
    timestamp: float = (msg.images[0].header.stamp.sec + msg.images[0].header.stamp.nanosec / 1e9)

    if first_stamp == -1:
        first_stamp = timestamp
        timestamp = 0
    else:
        timestamp = timestamp - first_stamp

    step['timestamp'] = timestamp
    step['seq'] = int(msg.images[0].header.frame_id)

    s += 1
    steps_data['steps'].append(step)

while reader.has_next():
    (topic, data, t) = reader.read_next()
    s += 1
    
df = pd.json_normalize(steps_data['steps'])
del steps_data

step_diff = df['seq'].max() - df['seq'].min()
fps = step_diff / df['timestamp'].max()
tmp_img = bridge.compressed_imgmsg_to_cv2(msg.images[0], desired_encoding="passthrough")

print("Start time: {}".format(datetime.datetime.fromtimestamp(first_stamp)))
print("Loaded {} steps from {}.".format(len(df.index), step_diff + 1))
print("Duration: {:.2f} seconds".format(df['timestamp'].max()))
print("Average FPS: {:.1f}".format(fps))
print("Action Space: {} actions".format(len(msg.results)))
print("Input image: {}x{}, {} channels.".format(tmp_img.shape[1], tmp_img.shape[0], tmp_img.shape[2]))
print("Total messages: {}".format(s))

In [ ]:
reader = rosbag2_py.SequentialReader()
reader.open(storage_options, converter_options)
reader.set_filter(storage_filter)

first_stamp = -1
steps_data = {'steps': []}

action_names = []
with open(metadata_json,"r") as jsonin:
    model_metadata=json.load(jsonin)
for action in model_metadata['action_space']:
    action_names.append(str(action['steering_angle'])+ u'\N{DEGREE SIGN}' + " "+"%.1f"%action["speed"])

writer = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*CODEC), fps, (width, height))
fig = create_plot(action_names, height, width, 72)

s = 0
with model.session as sess:

    cam = GradCam(model, model.get_conv_outputs())
    pbar = tqdm(total=s, desc="Loading messages", unit="messages")

    while reader.has_next():
        step = {}
        
        (topic, data, t) = reader.read_next()
        msg = deserialize_message(data, InferResultsArray)

        # Timestamp
        timestamp: float = (msg.images[0].header.stamp.sec + msg.images[0].header.stamp.nanosec / 1e9)

        if first_stamp == -1:
            first_stamp = timestamp
            timestamp = 0
        else:
            timestamp = timestamp - first_stamp

        step['timestamp'] = timestamp
        step['seq'] = int(msg.images[0].header.frame_id)

        # Extract original image from first camera
        cv_img = bridge.compressed_imgmsg_to_cv2(msg.images[0], desired_encoding="passthrough")
        cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGRA2RGB)
        # step['img'] = cv_img
        
        # Find best OpenVINO Result
        step['openvino_action'] = {'action': -1, 'probability': -1}    
        step['openvino_results'] = []
        for r in msg.results:
            step['openvino_results'].append({'action': r.class_label, 'probability': r.class_prob})
            if r.class_prob > step['openvino_action']['probability']:
                step['openvino_action'] = {'action': r.class_label, 'probability': r.class_prob}

        # Process image with Tensorflow
        tf_result, grad_img = cam.process(cv_img)
        #step['gradcam_img'] = grad_img
        
        step['tf_action'] = {'action': -1, 'probability': -1}    
        step['tf_results'] = []
        for i, r in enumerate(tf_result):
            step['tf_results'].append({'action': i, 'probability': r})
            if r > step['tf_action']['probability']:
                step['tf_action'] = {'action': i, 'probability': r}

        # Results
        step['results'] = []

        # Plot image
        img = update_plot(fig, step, action_names, cv_img, grad_img)

        pbar.update(1)
        s += 1

        steps_data['steps'].append(step)

        writer.write(img)

    pbar.close()
writer.release()
plt.close(fig)


In [ ]:
df = pd.json_normalize(steps_data['steps'])
del steps_data

step_diff = df['seq'].max() - df['seq'].min()
fps = step_diff / df['timestamp'].max()
print("Start time: {}".format(datetime.datetime.fromtimestamp(msg.images[0].header.stamp.sec)))
print("Loaded {} steps from {}.".format(len(df.index), step_diff))
print("Duration: {:.2f} seconds".format(df['timestamp'].max()))
print("Average FPS: {:.1f}".format(fps))
print("Action Space: {} actions".format(len(msg.results)))

In [ ]:
df['action_agree'] = np.where(df['openvino_action.action'] == df['tf_action.action'], 1, 0)
df['action_diff'] = np.abs(df['openvino_action.action'] - df['tf_action.action'])
action_analysis = df[['timestamp','seq','openvino_action.action','openvino_action.probability','tf_action.action', 'tf_action.probability', 'action_agree', 'action_diff']]
action_analysis.describe()

In [ ]:
# Creating histogram
fig, ax = plt.subplots(1, 2, figsize =(15, 5), sharey=True)
ax[0].hist(df['openvino_action.probability'], bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
ax[1].hist(df['tf_action.probability'], bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
plt.show()